In [58]:

import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

images_folder = "../image_generation/split_images/"
subfolders = os.listdir(images_folder)
deforestation_dataframes = []
all_features = []
for i in range(len(subfolders)):
    deforestation_information_df = pd.DataFrame(columns=["image", "timestamp", "x_coord", "y_coord", "label", "prev_label"])
    deforestation_dataframes.append(deforestation_information_df)
    
# Load a pretrained ResNet model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode (important for inference)

# Preprocess the image (resize, normalize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image
    transforms.CenterCrop(224),  # Crop the image to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Function to extract features
def extract_features(image_path):
    image = Image.open(image_path)  # Open image
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension

    with torch.no_grad():  # Disable gradient computation
        features = model(image)  # Get the features (model output)
    
    return features.numpy().flatten()  # Flatten the features to a 1D array

# Example: Extract features from an image
for i, folder in enumerate(subfolders):
    feature_vectors = []
    for image in os.listdir(images_folder +  folder):
        print(image)
        image_path = images_folder + folder + '/' + image
        features = list(extract_features(image_path))
        feature_vectors.append(features)
        data = {"image": image}
        deforestation_dataframes[i].loc[len(deforestation_dataframes[i].index)] = data 
    all_features.append(feature_vectors)
deforestation_dataframes
    


/Users/maanders1/miniconda3/envs/junction/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/maanders1/miniconda3/envs/junction/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_2.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_0.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_1.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_3.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_1.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_3.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_2.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_0.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_5_0.png
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_4.png
2020-07-01-00_00_2020-07-01-23

[                                                image  timestamp  x_coord  \
 0   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 1   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 2   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 3   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 4   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 5   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 6   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 7   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 8   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 9   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 10  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        NaN      NaN   
 11  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...        Na

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import pickle
import numpy as np


In [60]:
# features = []
# for df in deforestation_dataframes:
#     features.append(df["features"])
#     df.drop(labels=["features"], axis=1)

# print(f"Testing dataset: {len(features[0])} samples with {len(features[0][0])} features.")

In [61]:

with open('classifier.pkl', 'rb') as f:
    svm = pickle.load(f)

# Step 4: Evaluate the model on the test set
pd.set_option("display.max_rows", None)
for i, feature_vectors in enumerate(all_features):
    labels = svm.predict(feature_vectors)
    for j, label in enumerate(labels):
        image_name = deforestation_dataframes[i]["image"][j].split('/')[-1]
        print(image_name)
        timestamp = image_name.split("_S")[0]
        x_coord = image_name.split('_')[-2]
        y_coord = (image_name.split('_')[-1]).split('.')[0]
        print(x_coord, y_coord)
        if i != 0:
            previous_label = deforestation_dataframes[i - 1]["label"][j]
        else:
            previous_label = "not_deforestation"
        
        # data = {"image": image_name,
        #         "timestamp": timestamp,
        #         "x_coord": x_coord,
        #         "y_coord": y_coord,
        #         "label": label,
        #         "previous_label": previous_label
        #         }
        deforestation_dataframes[i].loc[j, ["image", "timestamp", "x_coord", "y_coord", "label", "prev_label"]] = [image_name, timestamp, x_coord, y_coord, label, previous_label] 

# Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred, pos_label="deforestation")
# precision = precision_score(y_test, y_pred, pos_label="deforestation")
# print(f'Accuracy: {accuracy * 100:.2f}%')
# print(f'Recall: {recall * 100:.2f}%')
# print(f'Precision: {precision * 100:.2f}%')
display(deforestation_dataframes)



2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_2.png
1 2
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_0.png
3 0
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_1.png
3 1
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_3.png
1 3
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_1.png
1 1
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_3.png
3 3
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_3_2.png
3 2
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_0.png
1 0
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_5_0.png
5 0
2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless_part_1_

/var/folders/hp/pczt0qkj68j3wh87bz8_662r0000gp/T/ipykernel_65358/4028701879.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2020-07-01-00_00_2020-07-01-23_59' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  deforestation_dataframes[i].loc[j, ["image", "timestamp", "x_coord", "y_coord", "label", "prev_label"]] = [image_name, timestamp, x_coord, y_coord, label, previous_label]
/var/folders/hp/pczt0qkj68j3wh87bz8_662r0000gp/T/ipykernel_65358/4028701879.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  deforestation_dataframes[i].loc[j, ["image", "timestamp", "x_coord", "y_coord", "label", "prev_label"]] = [image_name, timestamp, x_coord, y_coord, label, previous_label]
/var/folders/h

[                                                image  \
 0   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 1   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 2   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 3   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 4   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 5   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 6   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 7   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 8   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 9   2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 10  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 11  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 12  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 13  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 14  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 15  2020-07-01-00_00_2020-07-01-23_59_Sentinel-2_Q...   
 16  2020-07-0